In [4]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display_functions import display
from IPython.display import clear_output
from ipywidgets import IntSlider, interactive, FloatSlider, Output


class PoissonProcessSimulator:
    def __init__(self):
        self.interval = IntSlider(value=100, min=20, max=150, step=1, description="Time")
        self.param = FloatSlider(value=1, min=0.1, max=2, step=0.1, description="λ")

        self.interactive_plot = interactive(
            self.update_plot,
            interval=self.interval,
            param=self.param,
        )

    @staticmethod
    def simulate_process(interval: int, param: float):
        segments = []
        number = 0
        while True:
            number += 1
            start_time = 0 if not segments else segments[-1][0][1]
            end_time = start_time + np.random.exponential(param)
            if end_time > interval:
                break

            segments.append([(start_time, end_time), (number, number)])
        return segments

    def update_plot(self, interval: int, param: float):
        clear_output(wait=True)
        output_plot = Output()
        with output_plot:
            plt.figure(figsize=(10, 6))
            for data in self.simulate_process(interval, param):
                plt.plot(*data)

            plt.xlabel('Time')
            plt.ylabel('N')
            plt.title(f'Poisson Process λ = {param}')
            plt.show()
        display(output_plot)


if __name__ == "__main__":
    simulator = PoissonProcessSimulator()
    display(simulator.interactive_plot)


interactive(children=(IntSlider(value=100, description='Time', max=150, min=20), FloatSlider(value=1.0, descri…

In [1]:
from collections import deque
from enum import Enum

import numpy as np
from IPython.core.display_functions import display
from ipywidgets import Output
from matplotlib import pyplot as plt, interactive
from pydantic import BaseModel


class OrderAction(Enum):
    buy: str = "купить"
    sell: str = "продать"


class OrderType(Enum):
    market: str = "market"
    limit: str = "limit"


class Order(BaseModel):
    symbol: str
    action: OrderAction
    type: OrderType
    quantity: int
    price: int

    @staticmethod
    def generate_data(symbol: str = "BTC") -> dict:
        _action = np.random.choice([a for a in OrderAction], p=[0.5, 0.5])
        _type = np.random.choice([t for t in OrderType], p=[0.5, 0.5])
        _quantity = int(np.random.normal(10, 4))
        _price = int(np.random.normal(100, 10))
        return {
            "symbol": symbol,
            "action": _action,
            "type": _type,
            "quantity": _quantity,
            "price": _price,
        }


class OrderBookForSymbol(BaseModel):
    symbol: str = "BTC"
    buy: deque[Order] = deque()
    sell: deque[Order] = deque()

    def display_book(self):
        buy_prices = [order.price for order in self.buy]
        sell_prices = [order.price for order in self.sell]
        buy_quantities = [order.quantity for order in self.buy]
        sell_quantities = [order.quantity for order in self.sell]
        output_plot = Output()
        with output_plot:
            fig, ax = plt.subplots(1, 2, figsize=(12, 5))

            ax[0].barh(buy_prices, buy_quantities, color='g', alpha=0.7, label='Buy Orders')
            ax[0].set_xlabel('Quantity')
            ax[0].set_ylabel('Price')
            ax[0].set_title('Buy Orders')
            ax[0].legend()

            ax[1].barh(sell_prices, sell_quantities, color='r', alpha=0.7, label='Sell Orders')
            ax[1].set_xlabel('Quantity')
            ax[1].set_ylabel('Price')
            ax[1].set_title('Sell Orders')
            ax[1].legend()

            plt.suptitle(f"Order Book for {self.symbol}")
            plt.tight_layout()
            plt.show()
        display(output_plot)


if __name__ == "__main__":
    order_book = OrderBookForSymbol()

    for _ in range(5):
        order_book.buy.append(Order(**Order.generate_data()))

    for _ in range(5):
        order_book.sell.append(Order(**Order.generate_data()))

    order_book.display_book()


Output()